In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import logging
from sklearn.manifold import TSNE
import umap
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score



## Dataset

For this project, we will use the [Amazon Fine Food Reviews](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews/) dataset. It contains reviews of fine foods from Amazon, including text reviews and ratings. Here's the list of columns in the dataset:

- `Id`: Unique identifier for each review
- `ProductId`: Unique identifier for the product
- `UserId`: Unique identifier for the user
- `ProfileName`: Name of the user profile
- `HelpfulnessNumerator`: Number of helpful votes
- `HelpfulnessDenominator`: Total number of votes
- `Score`: Rating given by the user (1 to 5)
- `Time`: Timestamp of the review
- `Summary`: Summary of the review
- `Text`: Full text of the review

In [ ]:
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("averaged_perceptron_tagger", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("omw-1.4", quiet=True)

In [ ]:
logger = logging.getLogger(__name__)
random_state = 42
dataset_sample_size = 10000

In [ ]:
from typing import List, Union
import html
import re
class WordTokenizer:
    """
    A class for tokenizing and preprocessing text data using NLTK.
    This class handles text normalization, punctuation removal, and optional stop word filtering.

    Attributes:
        remove_stopwords (bool): Whether to remove stop words.
        lower_case (bool): Whether to convert text to lowercase.
    """

    def __init__(
        self,
        remove_stopwords: bool = True,
        lower_case: bool = True,
        use_lemmatization: bool = True,
        replace_repeated_chars: bool = True,
    ):
        self.remove_stopwords = remove_stopwords
        self.lower_case = lower_case
        self.lematizer = nltk.WordNetLemmatizer() if use_lemmatization else None
        self.stop_words = set(stopwords.words("english")) if remove_stopwords else set()
        self.replace_repeated_chars = replace_repeated_chars

    def process(
        self, text: Union[str, List[str]], return_tokens: bool = False
    ) -> Union[str, List[str]]:
        """
        Process the input text by cleaning, normalizing, and optionally removing stop words.

        Args:
            text (str | List[str]): The input text. If a list is provided, its elements will be joined into a single string.
            return_tokens (bool): If True, returns a list of tokens instead of a string.

        Returns:
            str | List[str]: The processed text as a string or list of tokens.
        """
        if not text:
            return [] if return_tokens else ""
        logger.debug(f"Processing text: {text}")

        text_str = " ".join(map(str, text)) if isinstance(text, list) else str(text)

        text = self._clean_text(text_str)
        logger.debug(f"Cleaned text: {text}")

        tokens = self._tokenize(text)
        logger.debug(f"Tokenized text: {tokens}")

        tokens = self._remove_stopwords(tokens)
        logger.debug(f"Tokens after stop word removal: {tokens}")

        tokens = self._apply_lemmatization(tokens)
        logger.debug(f"Tokens after lemmatization: {tokens}")

        tokens = [token for token in tokens if token]  # Remove empty tokens

        logger.debug(f"Final tokens: {tokens}")

        return tokens if return_tokens else " ".join(tokens)

    def _clean_text(self, text: str) -> str:
        if self.lower_case:
            text = text.lower()

        text = html.unescape(text)  # decode &amp; and other HTML entities
        text = text.strip().replace("\n", " ").replace("\r", " ")
        text = re.sub(r"<[^>]*>", "", text)  # Remove HTML tags
        text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
        text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
        text = self._fix_contractions(text)  # Fix common contractions

        # reduce repeated characters like "sooo good" to "so good" or "loooove" to "love"
        if self.replace_repeated_chars:
            text = re.sub(r"(.)\1{2,}", r"\1\1", text)

        return text

    def _apply_lemmatization(self, tokens: List[str]) -> List[str]:
        if self.lematizer is not None:
            tokens = [self.lematizer.lemmatize(token) for token in tokens]
        return tokens

    def _tokenize(self, text: str) -> List[str]:
        return word_tokenize(text)

    def _remove_stopwords(self, tokens: List[str]) -> List[str]:
        if self.remove_stopwords:
            tokens = self._remove_stop_words(tokens)
        return tokens

    def _remove_stop_words(self, tokens: List[str]) -> List[str]:
        return [word for word in tokens if word not in self.stop_words]

    def _fix_contractions(self, text: str) -> str:
        contraction_map = {
            "dont": "do not",
            "cant": "can not",
            "wont": "will not",
            "isnt": "is not",
            "arent": "are not",
            "wasnt": "was not",
            "werent": "were not",
            "didnt": "did not",
            "doesnt": "does not",
            "havent": "have not",
            "hasnt": "has not",
            "hadnt": "had not",
            "shouldnt": "should not",
            "wouldnt": "would not",
            "couldnt": "could not",
            "mustnt": "must not",
            "mightnt": "might not",
            "neednt": "need not",
            "im": "i am",
            "ive": "i have",
            "ill": "i will",
            "id": "i would",
            "youre": "you are",
            "youve": "you have",
            "youll": "you will",
            "youd": "you would",
            "theyre": "they are",
            "theyve": "they have",
            "theyll": "they will",
            "theyd": "they would",
            "weve": "we have",
            "theres": "there is",
            "heres": "here is",
            "thats": "that is",
            "whats": "what is",
            "whos": "who is",
            "don't": "do not",
            "can't": "can not",
            "won't": "will not",
            "isn't": "is not",
            "aren't": "are not",
            "wasn't": "was not",
            "weren't": "were not",
            "didn't": "did not",
            "doesn't": "does not",
            "haven't": "have not",
            "hasn't": "has not",
            "hadn't": "had not",
            "shouldn't": "should not",
            "wouldn't": "would not",
            "couldn't": "could not",
            "mustn't": "must not",
            "mightn't": "might not",
            "needn't": "need not",
            "i'm": "i am",
            "i've": "i have",
            "i'll": "i will",
            "i'd": "i would",
            "you're": "you are",
            "you've": "you have",
            "you'll": "you will",
            "you'd": "you would",
            "they're": "they are",
            "they've": "they have",
            "they'll": "they will",
            "they'd": "they would",
            "we've": "we have",
        }
        for contraction, full_form in contraction_map.items():
            text = re.sub(r"\b" + re.escape(contraction) + r"\b", full_form, text)
        return text

    def __call__(
        self, text: Union[str, List[str]], return_tokens: bool = False
    ) -> Union[str, List[str]]:
        """
        Allow the object to be called like a function.

        Example:
            tokenizer = WordTokenizer()
            clean_text = tokenizer("Some input text here.")
        """
        return self.process(text, return_tokens=return_tokens)


In [ ]:
def top_tfidf_words_per_cluster(tfidf_matrix, labels, vectorizer, cluster, n=10):
    indices = np.where(labels == cluster)[0]
    # Compute mean vector for the cluster
    cluster_tfidf = tfidf_matrix[indices].mean(axis=0)
    # Convert to 1D array if needed
    if hasattr(cluster_tfidf, 'A1'):
        cluster_tfidf = cluster_tfidf.A1
    else:
        cluster_tfidf = np.array(cluster_tfidf).flatten()
    # Get top n indices
    top_n = np.argsort(cluster_tfidf)[::-1][:n]
    res = [vectorizer.get_feature_names_out()[i] for i in top_n]
    return res

In [ ]:
def silhouette_scorer(estimator, X):
    """
    Custom scoring function for unsupervised clustering.
    Fits estimator on X and returns silhouette score.
    """
    labels = estimator.fit_predict(X)
    return silhouette_score(X, labels)

In [ ]:
# Uncomment this line if you have a pre-vectorized or pre-tokenized dataset and use it instead of the next line.
# df = pd.read_csv("./data/vectorized_reviews.csv") 

df = pd.read_csv("./data/Reviews.csv")
if dataset_sample_size > 0:
    df = df.sample(n=dataset_sample_size, random_state=random_state)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
grouped_by_product = df.groupby("ProductId").size().sort_values(ascending=False)
grouped_by_user = df.groupby("UserId").size().sort_values(ascending=False)

In [ ]:
sns.set_theme(style="whitegrid")

fig, axes = plt.subplots(3, 2, figsize=(16, 18))
fig.suptitle("FineFoodReview Dataset Analysis with Mean & Median", fontsize=18)

# 1. Histogram of reviews per user
mean_u = grouped_by_user.mean()
median_u = grouped_by_user.median()
sns.histplot(grouped_by_user.to_numpy(), bins=50, ax=axes[0,0], color="skyblue", kde=False)
axes[0,0].axvline(mean_u, color="red", linestyle="--", label=f"Mean: {mean_u:.2f}")
axes[0,0].axvline(median_u, color="green", linestyle="-.", label=f"Median: {median_u}")
axes[0,0].set_title("Distribution of Number of Reviews per User")
axes[0,0].set_xlabel("Number of Reviews")
axes[0,0].set_ylabel("Number of Users")
axes[0,0].legend()

# 2. Histogram of reviews per product
mean_p = grouped_by_product.mean()
median_p = grouped_by_product.median()
sns.histplot(grouped_by_product.to_numpy(), bins=50, ax=axes[0,1], color="lightcoral", kde=False)
axes[0,1].axvline(mean_p, color="red", linestyle="--", label=f"Mean: {mean_p:.2f}")
axes[0,1].axvline(median_p, color="green", linestyle="-.", label=f"Median: {median_p}")
axes[0,1].set_title("Distribution of Number of Reviews per Product")
axes[0,1].set_xlabel("Number of Reviews")
axes[0,1].set_ylabel("Number of Products")
axes[0,1].legend()

# 3. CDF for users
sns.ecdfplot(grouped_by_user.to_numpy(), ax=axes[1,0], color="blue")
axes[1,0].set_title("CDF of Number of Reviews per User")
axes[1,0].set_xlabel("Number of Reviews")
axes[1,0].set_ylabel("Cumulative Fraction of Users")
axes[1,0].grid(True)


# 4. CDF for products
sns.ecdfplot(grouped_by_product.to_numpy(), ax=axes[1,1], color="darkorange")
axes[1,1].set_title("CDF of Number of Reviews per Product")
axes[1,1].set_xlabel("Number of Reviews")
axes[1,1].set_ylabel("Cumulative Fraction of Products")
axes[1,1].grid(True)

# 5. Boxplot for users
sns.boxplot(x=grouped_by_user, ax=axes[2,0], color="lightblue", showmeans=True,
            meanprops={"marker":"o","markerfacecolor":"red","markeredgecolor":"black"})
axes[2,0].set_title("Boxplot of Number of Reviews per User")
axes[2,0].set_xlabel("Number of Reviews")

# 6. Boxplot for products
sns.boxplot(x=grouped_by_product, ax=axes[2,1], color="salmon", showmeans=True,
            meanprops={"marker":"o","markerfacecolor":"red","markeredgecolor":"black"})
axes[2,1].set_title("Boxplot of Number of Reviews per Product")
axes[2,1].set_xlabel("Number of Reviews")

plt.tight_layout(rect=(0, 0, 1, 0.97))
plt.show()




From the user and product statistics, we can see that the average number of reviews per user is around 2.22, with a maximum of 448 reviews by a single user. For products, the average number of reviews is about 7.66, with a maximum of 913 reviews for a single product. This indicates that while most users review only a few products, some users are very active in reviewing multiple products. Also, looking at the 75%, we can see that 75% of users have reviewed 2 or fewer products, and 75% of products have received 5 or fewer reviews. This suggests a long tail distribution where a few users and products are highly active while the majority are not. It's important to consider these statistics when training models, as they can impact the model's ability to generalize and the importance of certain users or products in the dataset. 


In [ ]:
tokenizer = WordTokenizer(
            remove_stopwords=True,
            lower_case=True,
            use_lemmatization=True,
        )
def tokenize_reviews(input_df, tkz: WordTokenizer, forced = False):
    if forced or "TokenizedText" not in input_df.columns:
        input_df["TokenizedText"] = input_df["Text"].apply(lambda x: tkz(x, return_tokens=True))
    return input_df



In [ ]:
tokenized_df = tokenize_reviews(df, tokenizer)
df = tokenized_df.copy(deep=True)
df

In [ ]:
# store the new dataset with tokenized reviews to save time in future runs
df.to_csv("data/fine_food_reviews_tokenized.csv", index=False)

## TFIDF

In [ ]:
df['JoinedTokens'] = df['TokenizedText'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

X_tfidf = vectorizer.fit_transform(df['JoinedTokens'])

df['doc_vector_tfidf'] = list(X_tfidf.toarray())
df

In [ ]:
kmeans_tfidf = KMeans(n_clusters=5, random_state=random_state)
df['Cluster_TFIDF'] = kmeans_tfidf.fit_predict(X_tfidf)

In [ ]:
df.to_csv("data/vectorized_reviews.csv", index=False)
df

## Hyperparameter Tuning

To optimize the KMeans clustering, we will perform hyperparameter tuning primarily on the number of clusters (k). We use the Elbow Method and Silhouette Score to determine the optimal k. The Elbow Method looks for the point where the Within-Cluster Sum of Squares (WSS) starts to diminish, indicating diminishing returns on adding more clusters. The Silhouette Score measures how similar an object is to its own cluster compared to other clusters, with higher scores indicating better-defined clusters.

In [ ]:
X_tfidf_dense = X_tfidf.toarray()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV

pipe_km = Pipeline([
    ('svd', TruncatedSVD(random_state=random_state)),
    ('cluster', KMeans(random_state=random_state))
])

param_grid_km = {
    'svd__n_components': [50, 100, 200, 300],
    'cluster__n_clusters': list(range(2, 30))
}

grid_km = GridSearchCV(
    pipe_km,
    param_grid_km,
    scoring=silhouette_scorer,
    cv=[(
        np.arange(X_tfidf_dense.shape[0]),
        np.arange(X_tfidf_dense.shape[0])
    )],
    return_train_score=False
)

grid_km.fit(X_tfidf_dense)
best_params_km = grid_km.best_params_
# Best KMeans pipeline params: {'cluster__n_clusters': 29, 'svd__n_components': 300}
print(f"Best KMeans pipeline params: {best_params_km}")
df['Cluster_KMeans_Pipe'] = grid_km.best_estimator_.named_steps['cluster'].labels_


Best KMeans pipeline params: {'cluster__n_clusters': 29, 'svd__n_components': 300}


In [ ]:
pipe_agg = Pipeline([
    ('svd', TruncatedSVD(random_state=random_state)),
    ('cluster', AgglomerativeClustering(linkage='ward'))
])
param_grid_agg = {
    'svd__n_components': [50, 100, 200, 300],
    'cluster__n_clusters': list(range(2, 30))
}
grid_agg = GridSearchCV(
    pipe_agg,
    param_grid_agg,
    scoring=silhouette_scorer,
    cv=[(
        np.arange(X_tfidf_dense.shape[0]),
        np.arange(X_tfidf_dense.shape[0])
    )],
    return_train_score=False
)
grid_agg.fit(X_tfidf_dense)
best_params_agg = grid_agg.best_params_
# Best Agglomerative pipeline params: {'cluster__n_clusters': 16, 'svd__n_components': 50}
print(f"Best Agglomerative pipeline params: {best_params_agg}")

df['Cluster_Agg_Pipe'] = grid_agg.best_estimator_.named_steps['cluster'].labels_

Best Agglomerative pipeline params: {'cluster__n_clusters': 16, 'svd__n_components': 50}


In [ ]:
score_km_pipe = silhouette_score(X_tfidf_dense, df['Cluster_KMeans_Pipe'])
score_agg_pipe = silhouette_score(X_tfidf_dense, df['Cluster_Agg_Pipe'])
print(f"Pipeline KMeans Silhouette = {score_km_pipe:.4f}")
print(f"Pipeline Agglomerative Silhouette = {score_agg_pipe:.4f}")

In [ ]:
X_vis = PCA(n_components=2).fit_transform(X_tfidf_dense)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes[0].scatter(X_vis[:,0], X_vis[:,1], c=df['Cluster_KMeans_Pipe'], cmap='rainbow', alpha=0.6)
axes[0].set_title(f"KMeans Pipeline (n_clusters={best_params_km['cluster__n_clusters']}, n_pca={best_params_km['pca__n_components']})")
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')

axes[1].scatter(X_vis[:,0], X_vis[:,1], c=df['Cluster_Agg_Pipe'], cmap='rainbow', alpha=0.6)
axes[1].set_title(f"Agglomerative Pipeline (n_clusters={best_params_agg['cluster__n_clusters']}, n_pca={best_params_agg['pca__n_components']})")
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')

plt.tight_layout()
plt.show()

In [ ]:
def tune_kmeans(X_tune, cluster_range_tunning, random_state=random_state):
    best_score = -1
    best_k = 0
    scores = {}
    for k in cluster_range_tunning:
        km = KMeans(n_clusters=k, random_state=random_state)
        labels = km.fit_predict(X_tune)
        score = silhouette_score(X_tune, labels)
        scores[k] = score
        print(f"K={k}: Silhouette Score = {score}")
        if score > best_score:
            best_score, best_k = score, k
    print(f"Best K: {best_k} with Silhouette Score = {best_score:.4f}")
    return best_k, scores

In [ ]:
k_range_km = range(19, 25)  # Testing k from 19 to 24
best_k_km, k_scores_km = tune_kmeans(X_tfidf, k_range_km, random_state)
print(f"Optimal number of clusters (k): {best_k_km}")

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(k_range_km, k_scores_km.values(), marker='o')
plt.title('Silhouette Scores for Different k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.tight_layout()
plt.show()

Based on the plots, select the optimal k. For demonstration, if elbow is at 5 and silhouette peaks around there, we stick with 5.
If a different k looks better (e.g., higher silhouette), update accordingly. Here, we assume 5 is still reasonable but note any observations.

In [ ]:
kmeans_opt = KMeans(n_clusters=best_k_km, random_state=random_state)
df['Cluster_TFIDF_Opt'] = kmeans_opt.fit_predict(X_tfidf)

In [ ]:
X_tfidf_dense = X_tfidf.toarray()
X_tfidf_pca = PCA(n_components=2).fit_transform(X_tfidf_dense)

plt.figure(figsize=(8,6))
plt.scatter(X_tfidf_pca[:,0], X_tfidf_pca[:,1], c=df['Cluster_TFIDF_Opt'], cmap='rainbow', alpha=0.6)
plt.title("PCA of TF-IDF Clusters")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()


In [ ]:
# t-SNE 

# Run t-SNE on TFIDF vectors
tsne_tfidf = TSNE(n_components=2, random_state=random_state, perplexity=30, max_iter=1000, init='random')
X_tsne_tfidf = tsne_tfidf.fit_transform(X_tfidf)

# Plot TFIDF t-SNE
plt.figure(figsize=(8, 7))
plt.scatter(X_tsne_tfidf[:,0], X_tsne_tfidf[:,1], c=df['Cluster_TFIDF_Opt'], cmap='tab10', alpha=0.6)
plt.title("t-SNE of Clusters (TFIDF vectors)")
plt.xlabel("t-SNE dim 1")
plt.ylabel("t-SNE dim 2")
plt.tight_layout()
plt.show()


In [ ]:
# Plot only TFIDF UMAP
umap_tfidf = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=random_state)
X_umap_tfidf = umap_tfidf.fit_transform(X_tfidf)
plt.figure(figsize=(8, 7))
plt.scatter(X_umap_tfidf[:,0], X_umap_tfidf[:,1], c=df['Cluster_TFIDF_Opt'], cmap='tab10', alpha=0.6)
plt.title("UMAP of Clusters (TFIDF vectors)")
plt.xlabel("UMAP dim 1")
plt.ylabel("UMAP dim 2")
plt.tight_layout()
plt.show()


In [ ]:
def plot_dendrogram(original_data, method='ward', metric='euclidean', subset_size=1000):
    
    subset_size = 1000  # Adjust based on computational resources
    X_tfidf_subset = original_data[:subset_size].toarray()  # Convert to dense for linkage

    # Compute linkage matrix for dendrogram
    Z = linkage(X_tfidf_subset, method=method, metric=metric)

    # Plot Dendrogram
    plt.figure(figsize=(12, 6))
    dendrogram(Z, truncate_mode='level', p=5)  # Truncate for readability
    plt.title('Dendrogram for Hierarchical Clustering (Subset)')
    plt.xlabel('Sample Index')
    plt.ylabel('Distance (Ward)')
    plt.show()

In [ ]:
plot_dendrogram(X_tfidf, method='ward', metric='euclidean', subset_size=1000)

In [ ]:
def tune_agglomerative(X_tune, cluster_range_tunning, random_state=random_state):
    best_score = -1
    best_k_ = None
    scores_ = {}
    for k in cluster_range_tunning:
        agg_cluster = AgglomerativeClustering(n_clusters=k)
        labels = agg_cluster.fit_predict(X_tune)
        score = silhouette_score(X_tune, labels)
        scores_[k] = score
        print(f"K={k}: Silhouette Score = {score}")
        if score > best_score:
            best_score, best_k_ = score, k
    print(f"Best K: {best_k_} with Silhouette Score = {best_score:.4f}")
    return best_k_, scores_


In [ ]:
k_range_agg = range(2, 6)  # Testing k from 2 to 5
best_k_agg, k_scores_agg = tune_agglomerative(X_tfidf.toarray(), k_range_agg, random_state)
print(f"Optimal number of agglomerative clusters (k): {best_k_agg}")

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(k_range_agg, k_scores_agg.values(), marker='o')
plt.title('Silhouette Scores for Different k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.tight_layout()
plt.show()

In [ ]:
agg_cluster = AgglomerativeClustering(n_clusters=best_k_agg, linkage='ward', metric='euclidean')
df['Cluster_Hierarchical'] = agg_cluster.fit_predict(X_tfidf.toarray())

In [ ]:
X_pca_hier = PCA(n_components=2).fit_transform(X_tfidf.toarray())
plt.figure(figsize=(8,6))
plt.scatter(X_pca_hier[:,0], X_pca_hier[:,1], c=df['Cluster_Hierarchical'], cmap='rainbow', alpha=0.6)
plt.title("PCA of Hierarchical Clusters")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
tsne_hier = TSNE(n_components=2, random_state=random_state, perplexity=30, max_iter=1000, init='random')
X_tsne_hier = tsne_hier.fit_transform(X_tfidf)
plt.figure(figsize=(8, 7))
plt.scatter(X_tsne_hier[:,0], X_tsne_hier[:,1], c=df['Cluster_Hierarchical'], cmap='tab10', alpha=0.6)
plt.title("t-SNE of Hierarchical Clusters (TFIDF vectors)")
plt.xlabel("t-SNE dim 1")
plt.ylabel("t-SNE dim 2")
plt.tight_layout()
plt.show()

In [ ]:
umap_hier = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=random_state)
X_umap_hier = umap_hier.fit_transform(X_tfidf)
plt.figure(figsize=(8, 7))
plt.scatter(X_umap_hier[:,0], X_umap_hier[:,1], c=df['Cluster_Hierarchical'], cmap='tab10', alpha=0.6)
plt.title("UMAP of Hierarchical Clusters (TFIDF vectors)")
plt.xlabel("UMAP dim 1")
plt.ylabel("UMAP dim 2")
plt.tight_layout()
plt.show()

In [ ]:
for i in range(best_k_km):
    print(f"Cluster {i}:")
    print(f"KMeans Cluster: {top_tfidf_words_per_cluster(X_tfidf, df['Cluster_TFIDF_Opt'], vectorizer, i)}")
print("-----------------------------------------------------------")
for i in range(best_k_agg):
    print(f"Cluster {i}:")
    print(f"nHierarchical Cluster: {top_tfidf_words_per_cluster(X_tfidf, df['Cluster_Hierarchical'], vectorizer, i)}")

In [ ]:
from sklearn.metrics import davies_bouldin_score, adjusted_rand_score

sil_kmeans = silhouette_score(X_tfidf, df['Cluster_TFIDF_Opt'])
db_kmeans = davies_bouldin_score(X_tfidf.toarray(), df['Cluster_TFIDF_Opt'])
sil_hier = silhouette_score(X_tfidf, df['Cluster_Hierarchical'])
db_hier = davies_bouldin_score(X_tfidf.toarray(), df['Cluster_Hierarchical'])

ari = adjusted_rand_score(df['Cluster_TFIDF_Opt'], df['Cluster_Hierarchical'])

print(f"KMeans Silhouette Score: {sil_kmeans:.4f}")
print(f"Hierarchical Silhouette Score: {sil_hier:.4f}")
print(f"KMeans Davies-Bouldin Index: {db_kmeans:.4f}")
print(f"Hierarchical Davies-Bouldin Index: {db_hier:.4f}")
print(f"Adjusted Rand Index between KMeans and Hierarchical: {ari:.4f}")